In [17]:
!pip install pydub

In [18]:
!pip install pandas

In [19]:
import pandas as pd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import random
import shutil
import os
from tqdm.auto import tqdm


In [9]:
import csv

input_file_path = './data/LibriSpeech/SPEAKERS.TXT'  # Путь к вашему файлу с данными
output_file_path = 'SPEAKERS.csv' # Путь к файлу CSV, который будет создан

# Открываем исходный файл для чтения и целевой файл для записи
with open(input_file_path, 'r') as infile, open(output_file_path, 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    
    # Записываем заголовок в CSV файл
    writer.writerow(['ID', 'SEX', 'SUBSET', 'MINUTES', 'NAME'])
    
    # Проходим по каждой строке в исходном файле
    for line in infile:
        # Убираем начальные и конечные пробелы и символы переноса строки
        line = line.strip()
        # Пропускаем строки комментариев и пустые строки
        if not line or line.startswith(';'):
            continue
        
        # Разделяем строку по символу '|'
        parts = [part.strip() for part in line.split('|')]
        # Записываем данные в CSV
        writer.writerow(parts)

print("Файл успешно преобразован в CSV.")


Файл успешно преобразован в CSV.


In [12]:
speakers = pd.read_csv('SPEAKERS.csv', error_bad_lines=False)

/var/folders/0l/2sk0lzs1471b1g3s6vt108c00000gn/T/ipykernel_73688/4199440838.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  speakers = pd.read_csv('SPEAKERS.csv', error_bad_lines=False)
b'Skipping line 34: expected 5 fields, saw 7\n'


In [20]:
list(speakers[speakers.ID == 14].SEX)[0]

'F'

In [ ]:

# Загрузка аудиофайлов
speech = AudioSegment.from_file("./data/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.wav", format="wav")
noise = AudioSegment.from_file("path_to_noise_file.wav", format="wav")

# Разделение речи на слова
chunks = split_on_silence(speech, min_silence_len=200, silence_thresh=-40)

# Создание итоговых аудиофайлов
speech_with_noise = AudioSegment()
speech_with_silence = AudioSegment()

# Индексы, где будет добавлен шум
noise_indices = random.sample(range(len(chunks)), k=int(len(chunks) * 0.3))  # Добавляем шум в 30% случаев

for i, chunk in enumerate(chunks):
    if i in noise_indices:
        # Вырезаем случайный сегмент шума длительностью с текущий чанк
        start = random.randint(0, len(noise) - len(chunk))
        noise_segment = noise[start:start + len(chunk)]

        # Добавляем шум в аудио с речью
        speech_with_noise += chunk.overlay(noise_segment)

        # Добавляем тишину той же длительности, что и шум
        silence = AudioSegment.silent(duration=len(chunk))
        speech_with_silence += silence
    else:
        # Добавляем чистый чанк в оба файла
        speech_with_noise += chunk
        speech_with_silence += chunk

# Экспорт аудиофайлов
speech_with_noise.export("speech_with_noise.wav", format="wav")
speech_with_silence.export("speech_with_silence.wav", format="wav")


In [23]:
AudioSegment.from_file("./data/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.wav", format="wav")

# Разбиение шумов и текстов по полу

In [28]:
source_folder = './data/dataset/vs_release_16k/audio_16k'
target_folder = './data/noise'
for file_name in os.listdir(source_folder):
    if file_name.startswith('f'):
        source_audio_path = os.path.join(source_folder, file_name)
        target_audio_path = os.path.join(target_folder, 'female', file_name)
    elif file_name.startswith('m'):
        source_audio_path = os.path.join(source_folder, file_name)
        target_audio_path = os.path.join(target_folder, 'male', file_name)
    elif file_name.startswith('o'):
        source_audio_path = os.path.join(source_folder, file_name)
        target_audio_path = os.path.join(target_folder, 'other', file_name)

    # Копирование аудиофайла в папку назначения
    shutil.copy(source_audio_path, target_audio_path)
#     print(f'File {file_name} has been copied to {target_folder}.')

In [94]:
source_folder = './data/dataset/LibriSpeech/dev-clean'
target_folder = './data/speech'
for speaker_id in os.listdir(source_folder):
    try:
        speaker_sex = list(speakers[speakers.ID == int(speaker_id)].SEX)[0]
        for chapter in os.listdir(os.path.join(source_folder, speaker_id)):
            for file_name in os.listdir(os.path.join(source_folder, speaker_id, chapter)):
                if file_name.endswith('.wav'):
                    if speaker_sex == 'F':
                        source_audio_path = os.path.join(source_folder, speaker_id, chapter, file_name)
                        target_audio_path = os.path.join(target_folder, 'female', file_name)
                    elif speaker_sex == 'M':
                        source_audio_path = os.path.join(source_folder, speaker_id, chapter, file_name)
                        target_audio_path = os.path.join(target_folder, 'male', file_name)
                    elif speaker_sex == 'O' or len(speakers[speakers.ID == int(speaker_id)]) == 0:
                        source_audio_path = os.path.join(source_folder, speaker_id, chapter, file_name)
                        target_audio_path = os.path.join(target_folder, 'other', file_name)
                    shutil.copy(source_audio_path, target_audio_path)
    except:
        print(speaker_id)
        continue

    # Копирование аудиофайла в папку назначения
#     print(f'File {file_name} has been copied to {target_folder}.')

.DS_Store
174
84


# Подготовка датасета (добавление сторонних звуков)

In [106]:
source_folder = './data/speech'
noise_folder = './data/noise'

clean_result_folder = './data/clean_speech'
noised_result_folder = './data/noised_speech'

for sex in tqdm(os.listdir(source_folder)):
    if sex.startswith('.'):
        continue
    for speech_path in tqdm(os.listdir(os.path.join(source_folder, sex))):
        #         print(os.path.join(source_folder, sex, speech_path))
        if speech_path.startswith('.'):
            continue
        speech = AudioSegment.from_file(os.path.join(source_folder, sex, speech_path), format="wav")
        # Разделение речи на слова
        chunks = split_on_silence(speech, min_silence_len=100, silence_thresh=-40)
        #         print(len(chunks))
        # Загрузка аудиофайлов
        #             speech = AudioSegment.from_file("./data/dataset/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.wav", format="wav")
        #             noise = AudioSegment.from_file("./data/noise/male/m0001_0_cough.wav", format="wav")


        # for i in chunks:
        #     display(i)
        # Создание итоговых аудиофайлов
        speech_with_noise = AudioSegment.empty()
        speech_with_silence = AudioSegment.empty()

        # Индексы, где будет добавлен шум
        noise_indices = random.sample(range(len(chunks)), k=1 if len(chunks) == 1 else random.randint(1, min([3, len(chunks) - 1])))  # Добавляем шум в 30% случаев

        for i, chunk in enumerate(chunks):
            if i in noise_indices:
                noise = AudioSegment.from_file(os.path.join(noise_folder, sex, random.choice(os.listdir(os.path.join(noise_folder, sex)))), format="wav")
                # Вырезаем случайный сегмент шума длительностью с текущий чанк
                noise_len = len(noise) # random.randint(500, len(noise))
                #                 noise_start = random.randint(0, noise_len)
                #                 noise_segment = noise[noise_start:noise_start + noise_len]

                # Добавляем шум в аудио с речью
                speech_with_noise += noise # _segment

                # Добавляем тишину той же длительности, что и шум
                silence = AudioSegment.silent(duration=noise_len)
                speech_with_silence += silence

                speech_with_noise += chunk
                speech_with_silence += chunk
            else:
                # Добавляем чистый чанк в оба файла
                speech_with_noise += chunk
                speech_with_silence += chunk

        # Экспорт аудиофайлов
        speech_with_noise.export(os.path.join(noised_result_folder, f"{sex}_{speech_path}"), format="wav")
        speech_with_silence.export(os.path.join(clean_result_folder, f"{sex}_{speech_path}"), format="wav")


  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1272 [00:00<?, ?it/s]

  0%|          | 0/1339 [00:00<?, ?it/s]

# Подготовка датасета (добавление фоновых звуков)

In [21]:
import os
import librosa
import numpy as np
import soundfile as sf


In [51]:

def add_noise_to_voice(voice_path, noise_path, output_path, noise_ratio=0.5):
    """
    Наложение шума на голос.
    
    :param voice_path: путь до файла с голосом
    :param noise_path: путь до файла с шумом
    :param output_path: путь для сохранения результирующего файла
    :param noise_ratio: соотношение уровня шума к голосу (0.0 - только голос, 1.0 - одинаковый уровень)
    :return: None
    """
    # Загрузка голосового и шумового файлов
    voice, sr_voice = librosa.load(voice_path, sr=None)
    noise, sr_noise = librosa.load(noise_path, sr=None)
    
    # Проверка частот дискретизации и приведение их к одинаковому значению
    if sr_voice != sr_noise:
        raise ValueError("Частота дискретизации голосового файла и шума должны быть одинаковы.")
    
    # Если шум короче голоса, повторяем шум до длины голоса
    if len(noise) < len(voice):
        noise = np.tile(noise, int(np.ceil(len(voice) / len(noise))))
    
    # Обрезаем шум до длины голоса
    noise = noise[:len(voice)]
    
    # Нормализация сигналов
    voice = voice / np.max(np.abs(voice))  # Нормализация голоса
    noise = noise / np.max(np.abs(noise))  # Нормализация шума

    # Наложение шума на голос
    combined_signal = voice + noise_ratio * noise

    # Нормализация комбинированного сигнала
    combined_signal = combined_signal / np.max(np.abs(combined_signal))
    
    # Сохранение результата
    sf.write(output_path, combined_signal, sr_voice)
    # print(f"Файл сохранен как: {output_path}")

# Пример использования
# add_noise_to_voice('voice.wav', 'noise.wav', 'voice_with_noise.wav', noise_ratio=0.3)

In [52]:
source_folder = './data/clean_speech'
result_folder = './data/back_noised_speech'
noise_folder = './data/back_noise'

noise_16k_folders = list(filter(lambda folder: '16k' in folder, os.listdir('data/back_noise')))
noise_files = dict()
for folder in noise_16k_folders:
    if folder.startswith('.'):
        continue
    for innerfolder in os.listdir(os.path.join(noise_folder, folder)):  
        if innerfolder.startswith('.'):
            continue
        noise_files[innerfolder] = []
        for filename in os.listdir(os.path.join(noise_folder, folder, innerfolder)):
            if filename.startswith('.'):
                continue
            noise_files[innerfolder].append(os.path.join(noise_folder, folder, innerfolder, filename))
noise_files_all = [j for i in noise_files.values() for j in i]
print('Объем библиотеки шумов:', len(noise_files_all))
for key in noise_files.keys():
    print('\t', key, len(noise_files[key]))

Объем библиотеки шумов: 272
	 DWASHING 16
	 STRAFFIC 16
	 NPARK 16
	 NRIVER 16
	 OHALLWAY 16
	 SPSQUARE 16
	 TCAR 16
	 OMEETING 16
	 PRESTO 16
	 DLIVING 16
	 DKITCHEN 16
	 NFIELD 16
	 PSTATION 16
	 TBUS 16
	 TMETRO 16
	 PCAFETER 16
	 OOFFICE 16


In [54]:
for speech_path in tqdm(os.listdir(source_folder)):
    if speech_path.startswith('.'):
        continue
    noise_file = random.choice(noise_files_all)
    add_noise_to_voice(os.path.join(source_folder, speech_path), noise_file, os.path.join(result_folder, speech_path))
    

  0%|          | 0/2685 [00:00<?, ?it/s]

In [56]:
import IPython

In [57]:
display(IPython.display.Audio('data/clean_speech/female_1462-170138-0000.wav', rate=16000))
display(IPython.display.Audio('data/back_noised_speech/female_1462-170138-0000.wav', rate=16000))